In [70]:
import os

midi_directory = "Audio"

midi_files = [os.path.join(midi_directory, file) for file in os.listdir(midi_directory) if file.endswith(".midi") or file.endswith(".mid")]

print("List of MIDI Files:")
print(midi_files)


List of MIDI Files:
['Audio\\MIDI-Unprocessed_Chamber1_MID--AUDIO_07_R3_2018_wav--2.midi', 'Audio\\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1.midi', 'Audio\\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--3.midi', 'Audio\\MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi', 'Audio\\MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--2.midi', 'Audio\\MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--3.midi', 'Audio\\MIDI-Unprocessed_Chamber4_MID--AUDIO_11_R3_2018_wav--1.midi', 'Audio\\MIDI-Unprocessed_Chamber4_MID--AUDIO_11_R3_2018_wav--3.midi', 'Audio\\MIDI-Unprocessed_Chamber5_MID--AUDIO_18_R3_2018_wav--1.midi', 'Audio\\MIDI-Unprocessed_Chamber5_MID--AUDIO_18_R3_2018_wav--2.midi', 'Audio\\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--1.midi', 'Audio\\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--2.midi', 'Audio\\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--3.midi', 'Audio\\MIDI-Unprocessed_Recital1-3_MID--AUDIO_01_R1_2018_wav--1.midi

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from music21 import converter, note, stream
import torch
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

In [71]:
def extract_notes_from_midi(midi_file):
    midi_stream = converter.parse(midi_file)
    

    notes = []
    for part in midi_stream.parts:
        for element in part.flatten().notes:
            if isinstance(element, note.Note):
                notes.append({
                    'pitch': element.pitch.midi,
                    'duration': element.duration.quarterLength,
                    'velocity': element.volume.velocity
                })
            elif isinstance(element, note.Rest):
                notes.append({
                    'is_rest': True,
                    'duration': element.duration.quarterLength
                })
    
    return notes

example_midi_file = midi_files[0]
example_notes = extract_notes_from_midi(example_midi_file)

print("Example Notes:")
print(example_notes)


Example Notes:
[{'pitch': 57, 'duration': Fraction(1, 3), 'velocity': 40}, {'pitch': 57, 'duration': 0.25, 'velocity': 45}, {'pitch': 57, 'duration': Fraction(2, 3), 'velocity': 44}, {'pitch': 57, 'duration': 0.25, 'velocity': 42}, {'pitch': 57, 'duration': Fraction(1, 3), 'velocity': 47}, {'pitch': 57, 'duration': 0.25, 'velocity': 42}, {'pitch': 57, 'duration': Fraction(1, 3), 'velocity': 47}, {'pitch': 57, 'duration': 0.25, 'velocity': 44}, {'pitch': 57, 'duration': Fraction(1, 3), 'velocity': 41}, {'pitch': 57, 'duration': 0.25, 'velocity': 45}, {'pitch': 57, 'duration': 0.25, 'velocity': 51}, {'pitch': 57, 'duration': Fraction(1, 3), 'velocity': 51}, {'pitch': 57, 'duration': 0.25, 'velocity': 50}, {'pitch': 57, 'duration': 0.25, 'velocity': 38}, {'pitch': 57, 'duration': Fraction(1, 3), 'velocity': 46}, {'pitch': 57, 'duration': 0.25, 'velocity': 44}, {'pitch': 57, 'duration': Fraction(1, 3), 'velocity': 50}, {'pitch': 83, 'duration': Fraction(4, 3), 'velocity': 77}, {'pitch': 55

In [72]:
def notes_to_tensor(notes):
    tensor_data = []
    for note_info in notes:
        if 'is_rest' in note_info:
            tensor_data.append([0, note_info['duration'], 0])  # Using 0 for pitch and velocity for rests
        else:
            tensor_data.append([note_info['pitch'], note_info['duration'], note_info['velocity']])
    
    return torch.tensor(tensor_data, dtype=torch.float32)

example_tensor_data = notes_to_tensor(example_notes)

print("Example Tensor Data:")
print(example_tensor_data)


Example Tensor Data:
tensor([[57.0000,  0.3333, 40.0000],
        [57.0000,  0.2500, 45.0000],
        [57.0000,  0.6667, 44.0000],
        ...,
        [45.0000,  0.2500, 62.0000],
        [81.0000,  0.2500, 66.0000],
        [81.0000,  0.7500, 66.0000]])


In [73]:
print(all_data)

[tensor([[57.0000,  0.3333, 40.0000],
        [57.0000,  0.2500, 45.0000],
        [57.0000,  0.6667, 44.0000],
        ...,
        [45.0000,  0.2500, 62.0000],
        [81.0000,  0.2500, 66.0000],
        [81.0000,  0.7500, 66.0000]]), tensor([[81.0000,  2.0000, 54.0000],
        [66.0000,  0.7500, 26.0000],
        [62.0000,  2.2500, 32.0000],
        ...,
        [54.0000,  0.5000, 69.0000],
        [57.0000,  0.2500, 66.0000],
        [66.0000,  0.3333, 86.0000]]), tensor([[53.0000,  0.2500, 39.0000],
        [65.0000,  0.5000, 30.0000],
        [53.0000,  0.3333, 37.0000],
        ...,
        [73.0000,  0.2500, 36.0000],
        [69.0000,  0.3333, 14.0000],
        [76.0000,  0.2500,  6.0000]]), tensor([[67.0000,  1.6667, 52.0000],
        [72.0000,  0.3333, 67.0000],
        [78.0000,  3.0000, 65.0000],
        ...,
        [35.0000,  3.0000, 21.0000],
        [35.0000,  4.0000, 21.0000],
        [35.0000,  3.0000, 21.0000]]), tensor([[ 78.0000,   3.5000,  59.0000],
        [ 7

In [78]:
padded_data = pad_sequence(all_data, batch_first=True)

print("Padded Dataset Shape:")
print(padded_data.shape)


Padded Dataset Shape:
torch.Size([100, 7042, 3])


In [79]:
sequence_lengths = [seq.size(0) for seq in all_data]

padded_data = pad_sequence(all_data, batch_first=True)

packed_data = pack_padded_sequence(padded_data, sequence_lengths, batch_first=True, enforce_sorted=False)

print("Packed Dataset Shape:")
print(packed_data.data.shape)


Packed Dataset Shape:
torch.Size([258870, 3])


In [77]:
all_data = []

for midi_file in midi_files:
    notes = extract_notes_from_midi(midi_file)
    tensor_data = notes_to_tensor(notes)
    all_data.append(tensor_data)



In [80]:
train_size = int(0.8 * len(padded_data))
test_size = len(padded_data) - train_size

train_dataset, test_dataset = random_split(padded_data, [train_size, test_size])


In [82]:
batch_size = 64 
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [83]:
class MusicGenerator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MusicGenerator, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, seq_len):
        out, _ = self.rnn(x)
        out = self.fc(out)
        return out[:, :seq_len, :]  


In [84]:
model = MusicGenerator(input_size=3, hidden_size=64, output_size=3)
example_batch = torch.randn((32, 100, 3)) 
example_seq_len = 80  
output = model(example_batch, example_seq_len)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1000  # Adjust as needed
for epoch in range(num_epochs):
    for batch in train_loader:
        inputs = batch[:, :-1, :]
        targets = batch[:, 1:, :]

        seq_len = targets.size(1)
        outputs = model(inputs, seq_len)

        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()

    with torch.no_grad():
        for batch in test_loader:
            inputs = batch[:, :-1, :]
            targets = batch[:, 1:, :]

            seq_len = targets.size(1)
            outputs = model(inputs, seq_len)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')


Epoch [1/1000], Loss: 1089.2249755859375
Epoch [2/1000], Loss: 975.5732421875
Epoch [3/1000], Loss: 1199.2786865234375
Epoch [4/1000], Loss: 1098.62353515625
Epoch [5/1000], Loss: 984.1784057617188
Epoch [6/1000], Loss: 983.196533203125
Epoch [7/1000], Loss: 874.7027587890625
Epoch [8/1000], Loss: 1128.55078125
Epoch [9/1000], Loss: 1013.8436889648438
Epoch [10/1000], Loss: 690.4089965820312
Epoch [11/1000], Loss: 983.6677856445312
Epoch [12/1000], Loss: 883.0596923828125
Epoch [13/1000], Loss: 703.6620483398438
Epoch [14/1000], Loss: 803.7818603515625
Epoch [15/1000], Loss: 1277.9005126953125
Epoch [16/1000], Loss: 1013.6724243164062
Epoch [17/1000], Loss: 786.2858276367188
Epoch [18/1000], Loss: 877.1038208007812
Epoch [19/1000], Loss: 1004.8614501953125
Epoch [20/1000], Loss: 871.3341674804688
Epoch [21/1000], Loss: 740.6605224609375
Epoch [22/1000], Loss: 811.2470703125
Epoch [23/1000], Loss: 1033.039794921875
Epoch [24/1000], Loss: 757.3449096679688
Epoch [25/1000], Loss: 885.0739

In [86]:
print(model.state_dict)

<bound method Module.state_dict of MusicGenerator(
  (rnn): LSTM(3, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=3, bias=True)
)>


In [113]:
torch.manual_seed(42)
seed_sequence = torch.randn((1, 500, 3))
print(seed_sequence)
generated_sequence = model(seed_sequence, 1000)  
def sequence_to_stream(generated_sequence):
    generated_stream = stream.Score()
    for step in generated_sequence:
        if isinstance(step[0], (int, float)):
            if step[0] == 0:
                generated_stream.append(note.Rest(quarterLength=float(step[1])))
            else:  # Note
                generated_stream.append(note.Note(midi=int(step[0]), quarterLength=float(step[1])))
        else:
            pass
    return generated_stream

generated_stream = sequence_to_stream(generated_sequence.squeeze().detach().numpy())

generated_sequences = generated_sequence

def tensor_to_stream(tensor_sequence):
    generated_stream = stream.Score()
    for step in tensor_sequence.squeeze().detach().numpy():
        pitch, duration, velocity = step

        if pitch < 0: 
            element = note.Rest(quarterLength=float(duration))
        else:
            mapped_pitch = int((pitch * 24) + 60)
            element = note.Note(midi=mapped_pitch, quarterLength=float(duration))
            element.volume.velocity = int((velocity + 1) * 64)  

        generated_stream.append(element)

    return generated_stream

midi_stream = tensor_to_stream(generated_sequence)

midi_file_path = "generated_music10001.mid"
midi_stream.write('midi', fp=midi_file_path)

print(f"MIDI file saved at: {midi_file_path}")

tensor([[[ 1.9269,  1.4873,  0.9007],
         [-2.1055,  0.6784, -1.2345],
         [-0.0431, -1.6047, -0.7521],
         ...,
         [ 0.5238, -1.8759, -1.0305],
         [ 1.3371, -0.2887, -1.5334],
         [-0.1797, -0.6578, -1.2317]]])
MIDI file saved at: generated_music10001.mid


In [114]:
model_path = "imodel.pth"
torch.save(model.state_dict(), model_path)

In [ ]:
loaded_model = MusicGenerator(input_size, hidden_size, output_size)
loaded_model.load_state_dict(torch.load(model_path))